<a href="https://colab.research.google.com/github/super-onigiri-man/ITE3-2/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FM愛媛 ベストヒットランキング ランキング早見表の使い方**


1.   コード左上の再生ボタン「上から」押す（コードが実行されます）

2.   途中で見たい週を選択します（1:今週,2:先週,3:選択した週）

3.   1,2の場合は約30秒で結果が出力されます。

     （水曜日の12時にサイトに掲載されるため、水曜日の正午以前に1を選択すると先週のランキングが出ます）

     （日付がたまにバグるときがあります。日付を確認してください）

4.   3の場合はYYYY-MM-DDの順序で入力してください。

     （例:2023年11月28日→2023-11-28）

     （これを選択した場合、入力された一番近い発表日のランキングが出ます）

     （未来の日付を入力するとエラーになります）

**利用上の注意！**

・このコードを書き換えないでください。書き換えると動かなくなる可能性があります。

  （将来の僕より優秀なプログラマーのADの方、このクソコードのおファクタリングよろしくお願いします）

・連続で使用する場合は5分以上時間を開けてください。

  （連続でアクセスすると各サイトのサーバーが攻撃とみなし、アクセス制限をかけられる場合があります。）

勉強になるかわかりませんが、このコードに説明を書いています。参考にでもしてください。

言語:Python

使用した技術:スクレイピング（Webサイトから情報を取得してくる技術のこと）


In [30]:
!pip install jupysql duckdb-engine

In [31]:
# エクステンションを読み込む
%load_ext sql

# DuckDBのインメモリのDBに接続する
%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [32]:
%reload_ext sql

In [41]:
%%sql
SELECT * FROM music_master.csv LIMIT 3;

 * duckdb://
Done.


Title,Artist,Score,Last_rank,Last_number,On_chart


In [35]:
import requests
from bs4 import BeautifulSoup #スクレイピング（取得）用
import datetime #日付計算用
import pandas as pd #ランキング用（将来的に使用予定）

#scoreを元にしたオリコンシングル・オリコンデジタル・ビルボード総合ランキング
#準備中
# オリコン週間ランキング用
OriconWeekData = []
# オリコンデジタルランキング用
OriconDigitalData = []
# ビルボードランキング用
BillboardData = []


def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回 3:別の回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    # print(str(Oriconday) + "付けオリコン週間シングルランキング")


    for link, artist in zip(links, artist):
      # 10位までランクインしたものを配列に保存
      OriconWeekData.append([link.text,artist.text,"{:.1f}".format(score)])
      # %sql INSERT INTO music_master.csv (Title, Artist, Score) VALUES (link.text, artist.text, Score) ON CONFLICT(Title) DO UPDATE SET Score = Score;

      # 壊れたときの表示用
      # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
      #   print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      # else: #10位（２ケタの場合）なら
      #   print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3


    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
      OriconWeekData.append([link.text,artist.text,"{:.1f}".format(score)])
        # 壊れたときの表示用
      # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3

    # print(OriconWeekRank)



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
      # 壊れたときの表示用
        # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
        #     print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        # else: #10位（２ケタの場合）なら
        #     print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けビルボードJAPAN HOT100ランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      BillboardData.append([song,artist,format(score, '.1f')])
      # if i < 9:
      #   print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      # else:
      #   print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する


sel = int(selecter())

if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())
    print()
    BillboadRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    print(OriconWeekData)
    print()
    OriconDigitalRank(OriconLastWeek())
    print(OriconDigitalData)
    print()
    BillboadRank(OriconLastWeek())
    print(BillboardData)

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())
    print()
    BillboadRank(OriconSelectWeek())


見たい週を選択してください
1:今回 2:前回 3:別の回2
[['アンチコンフィチュール', '≠ME', '6.0'], ['バケツを被れ!', 'HKT48', '5.7'], ['Kissin’ Christmas(クリスマスだからじゃない)2023', '桑田佳祐&松任谷由実', '5.4'], ['承認欲求', '櫻坂46', '5.1'], ['Heart Diamond', 'SUPER☆GiRLS', '4.8'], ['Monopoly', '乃木坂46', '4.5'], ['人生遊戯', 'Sexy Zone', '4.2'], ['Believe it leap/Illusion', '杉本琢弥', '3.9'], ['君の居場所(Have a Good Time Here)', '竹内まりや', '3.6'], ['Mr.Showtime/箱庭のコラル', 'ワンダーランズ×ショウタイム', '3.3'], ['Not Nightmare Christmas(Find the Sunlight/今日みたいな日、僕ら/Pain is Beautiful)', 'YESUNG', '3.0'], ['あなたのいちばんめ', 'KissBee', '2.7'], ['ギフテッド', 'NEWS', '2.4'], ['シブヤ 午後6時', 'ICEx', '2.1'], ['ラストノートしか知らない', '=LOVE', '1.8'], ['Snowbud/BIGHOUSE', 'MAPA', '1.5'], ['VIOLET LINE', 'Roselia', '1.2'], ['I Wish', 'なにわ男子', '0.9'], ['僕らはいま交差点に立って', 'ひめもすオーケストラ', '0.6'], ['RED LINE/ライフ イズ ビューティフル!', 'アンジュルム', '0.3']]

2024-01-01付けオリコン週間デジタルシングルランキング
[['唱', 'Ado', '6.0'], ['Closer Than This', 'Jimin', '5.7'], ['SOULSOUP', 'Official髭男dism', '5.4'], ['花になって - Be a flower', '緑黄色社会', '5.1'], [

In [39]:
for i in len(OriconWeekData):
    %%sql

    INSERT INTO test_music_master.csv (Title, Artist, Score)
    VALUES (OriconWeekData[i][0], OriconWeekData[i][1], OriconWeekData[i][2])
    ON CONFLICT(Title) DO UPDATE SET Score = OriconWeekData{i}[2];

SyntaxError: ignored